In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas(ascii=True)

from rdkit import Chem
import seaborn as sns

from sklearn.cluster import AgglomerativeClustering, DBSCAN, SpectralClustering
from scipy.stats import ks_2samp, chisquare,  power_divergence

import tmap, os
from faerun import Faerun
from mhfp.encoder import MHFPEncoder
from rdkit.Chem import AllChem
from rdkit import DataStructs, Chem
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from sklearn.decomposition import PCA

import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df_bace = pd.read_csv('../data/bace.csv')
df_chembl = pd.read_csv('../data/bace_chembl.csv')
tasks = 'Class'

In [3]:
df = df_bace.append(df_chembl)
df = df.reset_index(drop=True)

/home/sxh/anaconda3/envs/rdkit2020/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
bace_idx = df.index[:len(df_bace)]
chembl_idx = df.index[len(df_bace): ]

print(len(bace_idx), len(chembl_idx))

1513 5720


In [5]:
dim = 2048
mols = [Chem.MolFromSmiles(s) for s in df.smiles]
ECFP4_fps = [AllChem.GetMorganFingerprintAsBitVect(x,2,dim) for x in tqdm(mols, ascii=True)]

ecfps = [tmap.VectorUchar(list(fp)) for fp in ECFP4_fps]
enc = tmap.Minhash(dim, seed = 2048)

lf = tmap.LSHForest(dim)
lf.batch_add(enc.batch_from_binary_array(ecfps))
lf.index()

100%|##########| 7233/7233 [00:01<00:00, 6472.74it/s]


In [6]:
x, y, s, t, gp = tmap.layout_from_lsh_forest(lf)
X = np.array([x,y]).T

In [7]:

def adj_list_to_matrix(adj_list):
    n = len(adj_list)
    adj_matrix = np.zeros((n,n))
    for i,c in enumerate(adj_list):
        for (j, weight) in c:
            adj_matrix[i, j] = weight    
    return adj_matrix

adj_csr = adj_list_to_matrix(gp.adjacency_list)

clustering  = AgglomerativeClustering(n_clusters = 30, connectivity = adj_csr,).fit(X)
# clustering= SpectralClustering(n_clusters = n_clusters, random_state = 2, n_init = 100).fit(X)
dft = pd.concat([pd.Series(clustering.labels_), pd.Series(x)], axis=1)
order_dict = dft.groupby(0)[1].apply(np.min).sort_values().argsort().to_dict()

clustering.labels_ = pd.Series(clustering.labels_).map(order_dict).values

pd.Series(clustering.labels_).value_counts()

/home/sxh/anaconda3/envs/rdkit2020/lib/python3.6/site-packages/sklearn/cluster/_agglomerative.py:247: UserWarning: the number of connected components of the connectivity matrix is 534 > 1. Completing it to avoid stopping the tree early.
  affinity='euclidean')


15    442
23    412
22    395
12    350
21    339
16    311
14    288
27    285
0     275
13    272
2     271
7     265
29    263
6     258
5     257
28    223
18    196
11    195
9     187
19    186
26    182
8     182
4     168
10    165
17    159
24    147
3     145
1     143
25    143
20    129
dtype: int64

In [8]:
s = pd.Series(clustering.labels_)

In [9]:
occ = pd.Series(clustering.labels_[bace_idx]).unique()
set(occ)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 23,
 26,
 27,
 28,
 29}

In [10]:
select = set(pd.Series(clustering.labels_).unique()) - set(occ)
select

{20, 22, 24, 25}

In [11]:
s.value_counts()[list(select)]

24    147
25    143
20    129
22    395
dtype: int64

In [18]:
novel_idx = pd.Series(clustering.labels_)[pd.Series(clustering.labels_).isin({22})].index
len(novel_idx)

395

In [19]:
novel_idx = df.iloc[novel_idx].ID.tolist()
novel_idx = df_chembl[df_chembl.ID.isin(novel_idx)].index

In [20]:
df_chembl_novel = df_chembl.iloc[novel_idx].reset_index(drop=True)
df_chembl_common = df_chembl[~df_chembl.index.isin(novel_idx)].reset_index(drop=True)

In [21]:
df_chembl_novel.to_csv('../data/bace_chembl_novel.csv')
df_chembl_common.to_csv('../data/bace_chembl_common.csv')

In [22]:
df_chembl_novel.Class.value_counts()

1    216
0    179
Name: Class, dtype: int64

In [23]:
df_chembl_common.Class.value_counts()

0    3248
1    2077
Name: Class, dtype: int64